In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)  #  link pandas to plotly and add the iplot method
import plotly.express as px

In [ ]:
df = pd.read_csv("/kaggle/input/house-rent-prediction-dataset/House_Rent_Dataset.csv")
df.head()

In [ ]:
df.info()

In [ ]:
counts=df['City'].value_counts()
counts.to_frame()

In [ ]:
city_count =df['City'].value_counts()
df_city = pd.DataFrame({'labels': city_count.index,'values': city_count.values})

df_city.iplot(kind='pie',labels='labels',values='values', title='RENT IN DIFFERENT CITIES')

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x=df['City'], y=df['Rent'],errorbar=None)


In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x=df['City'], y=df['Rent'],hue = df['Furnishing Status'], palette='viridis',
errorbar=None)

In [ ]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=df, x="Size", y="Rent",hue="City")

In [ ]:
max_rent = df["Rent"].max()
index_MR = df[df["Rent"]==max_rent].index
df = df.drop(index_MR)

In [ ]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=df, x="Size", y="Rent",hue="City",size="City", sizes=(20,200))

In [ ]:
from wordcloud import WordCloud, STOPWORDS

text = " ".join(Company for Company in df["Area Locality"])
word_cloud = WordCloud(width = 1600,
                       height = 800,
                       colormap = 'prism',
                       background_color = "white").generate(text)
plt.figure(figsize=(20, 10))
plt.imshow(word_cloud, interpolation = "gaussian")
plt.axis("off")
plt.show()

In [ ]:
df.head()

In [ ]:
categorical_variable = [cname for cname in df.columns if  df[cname].nunique() <10 and  df[cname].dtype=="object" ]
categorical_variable


In [ ]:
def one_hot_encoding(df,col):
    one = pd.get_dummies(df[col],prefix=col)
    df = df.drop(col,axis=1)
    df = df.join(one)
    return df

In [ ]:
for col in categorical_variable:
     df = one_hot_encoding(df,col)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
for col in df.columns:
    if df[col].dtype == bool:
        df[col] = df[col].astype(int)
        
df

In [ ]:
df.dtypes

In [ ]:
df['Posted On'] = pd.to_datetime(df["Posted On"])

In [ ]:
df

In [ ]:
df['day of week posted'] = df['Posted On'].dt.day_of_week

df['day posted'] = df['Posted On'].dt.day

df['month posted'] = df['Posted On'].dt.month

df['quarter poster'] = df['Posted On'].dt.quarter

In [ ]:
df.dtypes

In [ ]:
df =df.join(df['Floor'].str.split(" out of ", n=1, expand=True).rename(columns={0:'Floor Level', 1:'Floor Max'}))


In [ ]:
df

In [ ]:
df['Floor Level'].unique()

In [ ]:
df['Floor Level'].replace({'Ground': 0, 'Lower Basement': -1,'Upper Basement':0},inplace=True)

In [ ]:
df['Floor Level'].unique()

In [ ]:
df

In [ ]:
df = df.dropna()
df['Floor Level'] = df['Floor Level'].astype(int)
df['Floor Max'] = df['Floor Max'].astype(int)


In [ ]:
df.info()

In [ ]:
df.drop(["Posted On",'Floor','Area Locality'],axis=1,inplace=True)
df

In [ ]:
print(df.duplicated().sum())
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split

target = 'Rent'
X = df.loc[:,df.columns!=target]
y = df.loc[:,df.columns==target]

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.20, shuffle=True)

In [ ]:
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

n_estimators= 200

RF = RandomForestRegressor(n_estimators=n_estimators,n_jobs=-1)

RF.fit(x_train, y_train.values.ravel())

train_accuracy = RF.score(x_train, y_train.values.ravel())


test_accuracy = RF.score(x_test, y_test.values.ravel())

print(f"Training Accuracy: {train_accuracy}")
print(f"Testing Accuracy: {test_accuracy}")

In [ ]:
# Predicting the Test set results
y_pred= RF.predict(x_test) 
X_pred= RF.predict(x_train)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
plt.figure(figsize=(20, 10))
ax = sns.distplot(y_train, hist = False, color = 'r', label = 'actual values')
sns.distplot(X_pred, hist = False, color = 'b', label = 'fitted_values', ax = ax)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
plt.figure(figsize=(20, 10))
ax = sns.distplot(y_test, hist = False, color = 'r', label = 'actual values')
sns.distplot(y_pred, hist = False, color = 'b', label = 'fitted_values', ax = ax)